# agent-alpha Walkthrough

This notebook demonstrates the full process:

1. LLM hypothesis generation (LangChain agent)
2. LLM blueprint generation (LangChain agent + tools)
3. Deterministic blueprint -> AST compilation
4. Deterministic AST evaluation (`agent_alpha`)


In [ ]:
import os
import sys
from pathlib import Path

cwd = Path.cwd().resolve()
repo_root = cwd
if not (repo_root / "agent-alpha").exists():
    if (cwd.parent / "agent-alpha").exists():
        repo_root = cwd.parent
    elif (cwd.parent.parent / "agent-alpha").exists():
        repo_root = cwd.parent.parent

agent_root = repo_root / "agent-alpha"

if str(agent_root) not in sys.path:
    sys.path.insert(0, str(agent_root))

print("agent_root:", agent_root)


In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY is not set. Export it before running this step.")
# Set your API key before running.

MODEL_NAME = "gpt-5-mini"


In [ ]:
from agent_alpha.runner import load_synthetic_panel

panel = load_synthetic_panel(n_days=220, n_tickers=50, seed=7)
panel 


In [ ]:
from agent_alpha.workflow import AgentAlphaWorkflow

workflow = AgentAlphaWorkflow(
    model_name=MODEL_NAME,
    #temperature=0.1,
    periods=[1],
    max_attempts=2,
)

state = workflow.run(
    user_goal="Build a readable, robust mean-reversion factor from OHLCV.",
    panel=panel,
    max_attempts=2,
)


In [ ]:
import json

print("error:", state.get("error"))
print("hypothesis:\n", state.get("hypothesis"))
print("\nAST expression:\n", state.get("ast_expression"))
print("\nAST summary:\n", state.get("ast_summary"))
print("\nmetrics:\n", json.dumps(state.get("metrics", {}), indent=2))
print("\nblueprint:\n", json.dumps(state.get("blueprint_json", {}), indent=2))


In [ ]:
factor = state.get("factor")
if factor is not None:
    display(factor.dropna())
